In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from dosipy.utils.viz import fig_config, save_fig
from utils import *

In [ ]:
def take_a_look(xt, yt, zt):
    fig_config(latex=True, scaler=1, text_size=16)
    fig = plt.figure()
    ax = plt.axes(projection ='3d')
    ax.scatter(xt, yt, zt, s=15, color='k', depthshade=True)
    ax.plot(xs[:int(xs.size/2)-50],
            ys[:int(xs.size/2)-50],
            zs[:int(xs.size/2)-50],
            linestyle='-', c='k')
    ax.plot(xs[int(xs.size/2)+50:],
            ys[int(xs.size/2)+50:],
            zs[int(xs.size/2)+50:],
            linestyle='-', c='k')
    ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
           xticks=[xt.min(), 0.0, xt.max()],
           yticks=[h, h/2, 0.0],
           zticks=[zt.min(), 0.0, zt.max()],
           xticklabels=[int(round(xt.min()*1000, 1)),
                        0,
                        int(round(xt.max()*1000, 1))],
           yticklabels=['$d$',
                        '$d/2$',
                        0],
           zticklabels=[int(round(zt.min()*1000, 1)),
                        0,
                        int(round(zt.max()*1000, 1))],
          )
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.labelpad = 10
    ax.yaxis.labelpad = 10
    ax.zaxis.labelpad = 10
    ax.view_init(25, -45)
    fig.tight_layout()
    return fig, ax

In [ ]:
# frequency
f = 6

# separation distance
h = -2.5 / 1000

# averaging surface
if f < 30:
    target_area = (0.02, 0.02)
else:
    target_area = (0.01, 0.01)
A = target_area[0] * target_area[1]
target_area_origin = (-target_area[0]/2, -target_area[1]/2)

# source
xs = np.load(os.path.join('data', f'x_at{f}GHz.npy'))
xs = xs - xs.max() / 2
xs = np.asarray(xs.flatten())
ys = np.zeros_like(xs) + h
zs = np.zeros_like(xs)

In [ ]:
# planar target
N = 7
x = np.linspace(-target_area[0]/2, target_area[0]/2, N)
y = 0.0
z = np.linspace(-target_area[1]/2, target_area[1]/2, N)
Xt, Zt = np.meshgrid(x, z)
xt_pln = Xt.ravel()
yt_pln = np.zeros_like(xt_pln)
zt_pln = Zt.ravel()

fig, ax = take_a_look(xt_pln, yt_pln, zt_pln)

nx = 0
ny = -1
nz = 0
q = ax.quiver(xt_pln, yt_pln, zt_pln, nx, ny, nz,
              lw=1, length=np.abs(h)/3, arrow_length_ratio=0.3, color='k')

# fname = os.path.join('figures', 'planar_surface')
# save_fig(fig, fname, formats=['png'])

In [ ]:
# spherical target
N = 7
r = 0.12
alpha = 2 * np.arcsin(target_area[0]/2/r)
theta = np.linspace(np.pi/2-alpha/2, np.pi/2+alpha/2, N)
phi = np.linspace(np.pi-alpha/2, np.pi+alpha/2, N)
Theta, Phi = np.meshgrid(theta, phi)
yt_sph, xt_sph, zt_sph = sph2cart(r, Theta.ravel(), Phi.ravel())
yt_sph -= yt_sph.min()

fig, ax = take_a_look(xt_sph, yt_sph, zt_sph)

ny_sph, nx_sph, nz_sph = sph_normals(r, Theta.ravel(), Phi.ravel())
q = ax.quiver(xt_sph, yt_sph, zt_sph, nx_sph, ny_sph, nz_sph,
              lw=1, length=np.abs(h)/3, normalize=True, color='k')

# fname = os.path.join('figures', 'spherical_surface')
# save_fig(fig, fname, formats=['png'])

In [ ]:
# cylindrical target
N = 7
r = 0.0725
alpha = 2 * np.arcsin(target_area[0]/2/r)
theta = np.linspace(np.pi/2-alpha/2, np.pi/2+alpha/2, N)
Theta, Zt = np.meshgrid(-theta, z)
xt_cyl, yt_cyl, zt_cyl = cyl2cart(r, Theta.ravel(), Zt.ravel())
yt_cyl -= yt_cyl.min()

fig, ax = take_a_look(xt_cyl, yt_cyl, zt_cyl)

nx_cyl, ny_cyl, nz_cyl = cyl_normals(r, Theta.ravel(), zt_cyl)
q = ax.quiver(xt_cyl, yt_cyl, zt_cyl, nx_cyl, ny_cyl, nz_cyl,
              lw=1, length=np.abs(h)/3, normalize=True, color='k')

# fname = os.path.join('figures', 'cylindrical_surface')
# save_fig(fig, fname, formats=['png'])